In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from googletrans import Translator
from pandas import Series
from pandas import DataFrame
%matplotlib inline


In [2]:
item_category_df = pd.read_csv("data/item_categories.csv")
items_df = pd.read_csv("data/items.csv")
shops_df = pd.read_csv("data/shops.csv")
sample_df = pd.read_csv("data/sample_submission.csv")
train_df = pd.read_csv("data/sales_train_v2.csv")
test_df = pd.read_csv("data/test.csv")

# date-> day, month, year

In [3]:
f = lambda x: x.split(".")
train_df['date'] =list(map(f, train_df['date']))

In [4]:
f = lambda x: x[0]
train_df['day'] =list(map(f, train_df['date']))

In [5]:
f = lambda x: x[1]
train_df['month'] =list(map(f, train_df['date']))

In [6]:
f = lambda x: x[2]
train_df['year'] =list(map(f, train_df['date']))

In [7]:
train_df = train_df.drop(['date'],axis=1)

In [8]:
y_data = train_df.item_cnt_day.reshape(train_df.item_cnt_day.size,1)

C:\Users\my\Miniconda3\envs\ml_python\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.


In [9]:
train_df = train_df.drop(['item_cnt_day'],axis=1)

## 번역 (러시아어 -> 영어) -> tf-idf

In [10]:
trans = Translator()
def translate(column):
    en_list=[]
    for word in column:
        en = trans.translate(word, dest='en')
        en_list.append(en.text)
    return en_list

In [11]:
shops_df["shop_name"] = translate(shops_df["shop_name"])
shops_df.head()

,shop_name,shop_id
0,"Yakutsk Ordzhonikidze, 56 francs",0
1,"Yakutsk TC ""Central"" franc",1
2,"Adygea Shopping Center ""Mega""",2
3,"Balashikha TRK ""October-Kinomir""",3
4,"Volzhsky shopping center ""Volga Mall""",4


In [12]:
en_shop_name = np.asarray(shops_df.shop_name)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,4), min_df=0.03, max_df = 0.95)
shop_name_model = vectorizer.fit_transform(en_shop_name).toarray()

C:\Users\my\Miniconda3\envs\ml_python\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [14]:
shop_X =[]
for i in range(len(en_shop_name)):
    shop_X.append(shop_name_model[i])

In [15]:
shops_df["shop_name_tfidf"] = shop_X
shops_df.head()

,shop_name,shop_id,shop_name_tfidf
0,"Yakutsk Ordzhonikidze, 56 francs",0,"[0.416609635057569, 0.0, 0.0, 0.0, 0.0, 0.0, 0..."
1,"Yakutsk TC ""Central"" franc",1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.53929388..."
2,"Adygea Shopping Center ""Mega""",2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4538104235202..."
3,"Balashikha TRK ""October-Kinomir""",3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"Volzhsky shopping center ""Volga Mall""",4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.4320054028830..."


In [16]:
for i in range(len(shops_df)):
    for j in range(len(shops_df['shop_name_tfidf'][0])):
        shops_df['shop_name_tfidf' + str(j)]= shops_df['shop_name_tfidf'][i][j]

In [17]:
shops_df = shops_df.drop(['shop_name_tfidf'],axis=1)
shops_df.head()

,shop_name,shop_id,shop_name_tfidf0,shop_name_tfidf1,shop_name_tfidf2,shop_name_tfidf3,shop_name_tfidf4,shop_name_tfidf5,shop_name_tfidf6,shop_name_tfidf7,...,shop_name_tfidf65,shop_name_tfidf66,shop_name_tfidf67,shop_name_tfidf68,shop_name_tfidf69,shop_name_tfidf70,shop_name_tfidf71,shop_name_tfidf72,shop_name_tfidf73,shop_name_tfidf74
0,"Yakutsk Ordzhonikidze, 56 francs",0,0.0,0.0,0.0,0.0,0.0,0.0,0.546225,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Yakutsk TC ""Central"" franc",1,0.0,0.0,0.0,0.0,0.0,0.0,0.546225,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Adygea Shopping Center ""Mega""",2,0.0,0.0,0.0,0.0,0.0,0.0,0.546225,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Balashikha TRK ""October-Kinomir""",3,0.0,0.0,0.0,0.0,0.0,0.0,0.546225,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Volzhsky shopping center ""Volga Mall""",4,0.0,0.0,0.0,0.0,0.0,0.0,0.546225,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
item_category_df["item_category_name"] = translate(item_category_df["item_category_name"])
item_category_df.head()

,item_category_name,item_category_id
0,PC - Headsets / Headphones,0
1,Accessories - PS2,1
2,Accessories - PS3,2
3,Accessories - PS4,3
4,Accessories - PSP,4


In [19]:
en_item_category_name = np.asarray(item_category_df.item_category_name)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,4), min_df=0.03, max_df = 0.95)
item_category_name_model = vectorizer.fit_transform(en_item_category_name).toarray()

C:\Users\my\Miniconda3\envs\ml_python\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [21]:
item_category_X =[]
for i in range(len(en_item_category_name)):
    item_category_X.append(item_category_name_model[i])

In [22]:
item_category_df["item_category_tfidf"] = item_category_X
item_category_df.head()

,item_category_name,item_category_id,item_category_tfidf
0,PC - Headsets / Headphones,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Accessories - PS2,1,"[0.0, 0.6247350911517717, 0.0, 0.0, 0.0, 0.0, ..."
2,Accessories - PS3,2,"[0.0, 0.6247350911517717, 0.0, 0.0, 0.0, 0.0, ..."
3,Accessories - PS4,3,"[0.0, 0.6247350911517717, 0.0, 0.0, 0.0, 0.0, ..."
4,Accessories - PSP,4,"[0.0, 0.6247350911517717, 0.0, 0.0, 0.0, 0.0, ..."


In [23]:
for i in range(len(item_category_df)):
    for j in range(len(item_category_df['item_category_tfidf'][0])):
        item_category_df['item_category_tfidf' + str(j)]= item_category_df['item_category_tfidf'][i][j]

In [24]:
item_category_df = item_category_df.drop(['item_category_tfidf'],axis=1)
item_category_df.head()

,item_category_name,item_category_id,item_category_tfidf0,item_category_tfidf1,item_category_tfidf2,item_category_tfidf3,item_category_tfidf4,item_category_tfidf5,item_category_tfidf6,item_category_tfidf7,...,item_category_tfidf28,item_category_tfidf29,item_category_tfidf30,item_category_tfidf31,item_category_tfidf32,item_category_tfidf33,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37
0,PC - Headsets / Headphones,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Accessories - PS2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Accessories - PS3,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Accessories - PS4,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Accessories - PSP,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
items_df.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [26]:
rus_item_name = np.asarray(items_df.item_name)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,4), min_df=0.03, max_df = 0.95)
rus_item_name_model = vectorizer.fit_transform(rus_item_name).toarray()

C:\Users\my\Miniconda3\envs\ml_python\lib\site-packages\sklearn\feature_extraction\text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [28]:
item_X =[]
for i in range(len(rus_item_name)):
    item_X.append(rus_item_name_model[i])

In [29]:
items_df["item_tfidf"] = item_X
items_df.head()

,item_name,item_id,item_category_id,item_tfidf
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,"[0.0, 0.0, 0.0, 0.0, 0.4825704672175713, 0.0, ..."
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,***КОРОБКА (СТЕКЛО) D,4,40,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [30]:
for i in range(len(items_df)):
    for j in range(len(items_df['item_tfidf'][0])):
        items_df['item_tfidf' + str(j)]= items_df['item_tfidf'][i][j]

In [31]:
items_df = items_df.drop(['item_tfidf'],axis=1)
items_df.head()

,item_name,item_id,item_category_id,item_tfidf0,item_tfidf1,item_tfidf2,item_tfidf3,item_tfidf4,item_tfidf5,item_tfidf6,...,item_tfidf13,item_tfidf14,item_tfidf15,item_tfidf16,item_tfidf17,item_tfidf18,item_tfidf19,item_tfidf20,item_tfidf21,item_tfidf22
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,***КОРОБКА (СТЕКЛО) D,4,40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# merge

In [32]:
df =pd.merge(train_df, shops_df, on='shop_id')
df.head()

,date_block_num,shop_id,item_id,item_price,day,month,year,shop_name,shop_name_tfidf0,shop_name_tfidf1,...,shop_name_tfidf65,shop_name_tfidf66,shop_name_tfidf67,shop_name_tfidf68,shop_name_tfidf69,shop_name_tfidf70,shop_name_tfidf71,shop_name_tfidf72,shop_name_tfidf73,shop_name_tfidf74
0,0,59,22154,999.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,59,22151,399.0,10,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,59,5603,699.0,04,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,59,5587,199.0,19,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,59,5613,5571.0,31,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
df =pd.merge(df, items_df, on='item_id')
df.head()

,date_block_num,shop_id,item_id,item_price,day,month,year,shop_name,shop_name_tfidf0,shop_name_tfidf1,...,item_tfidf13,item_tfidf14,item_tfidf15,item_tfidf16,item_tfidf17,item_tfidf18,item_tfidf19,item_tfidf20,item_tfidf21,item_tfidf22
0,0,59,22154,999.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,25,22154,999.0,02,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,25,22154,999.0,03,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,25,22154,999.0,20,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,25,22154,999.0,23,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
df =pd.merge(df, item_category_df, on='item_category_id')
df.head()

,date_block_num,shop_id,item_id,item_price,day,month,year,shop_name,shop_name_tfidf0,shop_name_tfidf1,...,item_category_tfidf28,item_category_tfidf29,item_category_tfidf30,item_category_tfidf31,item_category_tfidf32,item_category_tfidf33,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37
0,0,59,22154,999.0,02,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,25,22154,999.0,02,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,25,22154,999.0,03,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,25,22154,999.0,20,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,25,22154,999.0,23,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
df = df.drop(['item_name'],axis=1)
df = df.drop(['item_category_name'],axis=1)

In [36]:
df = df.drop(['day'],axis=1)
# df = df.drop(['year'],axis=1)

In [37]:
df = df.drop(['item_price'],axis=1)

In [38]:
df.head()

,date_block_num,shop_id,item_id,month,year,shop_name,shop_name_tfidf0,shop_name_tfidf1,shop_name_tfidf2,shop_name_tfidf3,...,item_category_tfidf28,item_category_tfidf29,item_category_tfidf30,item_category_tfidf31,item_category_tfidf32,item_category_tfidf33,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37
0,0,59,22154,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,25,22154,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,25,22154,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,25,22154,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,25,22154,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# location feature


In [39]:
loc_shop = df["shop_name"]
len(loc_shop)

2935849

In [40]:
loc = []
for i in loc_shop:
    splited_loc = i.split()
    loc.append(splited_loc[0])

In [41]:
df['location'] = loc

In [42]:
west= ['Adygea','Balashikha', 'Chekhov','Kaluga','Kazan', 'Krasnoyarsk' ,'Kursk', 'Moscow' ,'Yaroslavl',
        'Mytischi', 'N.Novgorod', 'Nizhny','RostovNaDon',  'RostovNaDonu', 'Samara' ,'SPb' ,'Sergiev', 
        'Surgut', 'Tyumen', 'Ufa', 'Vologda', 'Volzhsky', 'Voronezh' , 'Zhukovsky','Colosseum', 'Khimki']
east =  ['Yakutsk']
south =  ['Novosibirsk','Omsk', 'Tomsk']

In [43]:
s1 = Series(data = 'west', index= west)
west = s1.to_dict()

s2 = Series(data = 'east'
   , index= east)
east = s2.to_dict()

s3 = Series(data = 'south'
   , index= south)
south = s3.to_dict()

In [44]:
df["location"].replace(to_replace=west.keys(), value=west.values(), inplace=True)
df["location"].replace(to_replace=east.keys(), value=east.values(), inplace=True)
df["location"].replace(to_replace=south.keys(), value=south.values(), inplace=True)
df.head()

,date_block_num,shop_id,item_id,month,year,shop_name,shop_name_tfidf0,shop_name_tfidf1,shop_name_tfidf2,shop_name_tfidf3,...,item_category_tfidf29,item_category_tfidf30,item_category_tfidf31,item_category_tfidf32,item_category_tfidf33,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37,location
0,0,59,22154,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,west
1,0,25,22154,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,west
2,0,25,22154,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,west
3,0,25,22154,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,west
4,0,25,22154,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,west


In [45]:
dummy_loc_df =pd.get_dummies(df['location'])
dummy_loc_df.head()

,Digital,Online,Outbound,east,south,west
0,0,0,0,0,0,1
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,0,0,1


In [46]:
df = pd.concat([df, dummy_loc_df],axis=1)
df = df.drop(['location'],axis=1)
df.head()

,date_block_num,shop_id,item_id,month,year,shop_name,shop_name_tfidf0,shop_name_tfidf1,shop_name_tfidf2,shop_name_tfidf3,...,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37,Digital,Online,Outbound,east,south,west
0,0,59,22154,01,2013,"Yaroslavl Shopping center ""Altair""",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
1,0,25,22154,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
2,0,25,22154,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
3,0,25,22154,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
4,0,25,22154,01,2013,"Moscow TRK ""Atrium""",0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1


In [47]:
# df = df.drop(['item_name'],axis=1)
df = df.drop(['item_category_id'],axis=1)
df = df.drop(['shop_name'],axis=1)

In [48]:
train_df = df
train_df.head()

,date_block_num,shop_id,item_id,month,year,shop_name_tfidf0,shop_name_tfidf1,shop_name_tfidf2,shop_name_tfidf3,shop_name_tfidf4,...,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37,Digital,Online,Outbound,east,south,west
0,0,59,22154,01,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
1,0,25,22154,01,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
2,0,25,22154,01,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
3,0,25,22154,01,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1
4,0,25,22154,01,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,0,1


In [49]:
train_df['ID'] = train_df.index
train_df.head()

,date_block_num,shop_id,item_id,month,year,shop_name_tfidf0,shop_name_tfidf1,shop_name_tfidf2,shop_name_tfidf3,shop_name_tfidf4,...,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37,Digital,Online,Outbound,east,south,west,ID
0,0,59,22154,01,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,1,0
1,0,25,22154,01,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,1,1
2,0,25,22154,01,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,1,2
3,0,25,22154,01,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,1,3
4,0,25,22154,01,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,1,4


# test

In [50]:
test_df = pd.read_csv("data/test.csv")

In [51]:
test_df.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [52]:
test_df =pd.merge(test_df, items_df, on='item_id')
# test_df.head()

In [53]:
test_df =pd.merge(test_df, shops_df, on='shop_id')
# test_df.head()

In [54]:
test_df =pd.merge(test_df, item_category_df, on='item_category_id')
# test_df.head()

In [55]:
loc_shop = test_df["shop_name"]
len(loc_shop)

214200

In [56]:
loc = []
for i in loc_shop:
    splited_loc = i.split()
    loc.append(splited_loc[0])

In [57]:
test_df['location'] = loc

In [58]:
west= ['Adygea','Balashikha', 'Chekhov','Kaluga','Kazan', 'Krasnoyarsk' ,'Kursk', 'Moscow' ,'Yaroslavl',
        'Mytischi', 'N.Novgorod', 'Nizhny','RostovNaDon',  'RostovNaDonu', 'Samara' ,'SPb' ,'Sergiev', 
        'Surgut', 'Tyumen', 'Ufa', 'Vologda', 'Volzhsky', 'Voronezh' , 'Zhukovsky','Colosseum', 'Khimki']
east =  ['Yakutsk']
south =  ['Novosibirsk','Omsk', 'Tomsk']

In [59]:
s1 = Series(data = 'west', index= west)
west = s1.to_dict()

s2 = Series(data = 'east'
   , index= east)
east = s2.to_dict()

s3 = Series(data = 'south'
   , index= south)
south = s3.to_dict()

In [60]:
test_df["location"].replace(to_replace=west.keys(), value=west.values(), inplace=True)
test_df["location"].replace(to_replace=east.keys(), value=east.values(), inplace=True)
test_df["location"].replace(to_replace=south.keys(), value=south.values(), inplace=True)
test_df.head()

,ID,shop_id,item_id,item_name,item_category_id,item_tfidf0,item_tfidf1,item_tfidf2,item_tfidf3,item_tfidf4,...,item_category_tfidf29,item_category_tfidf30,item_category_tfidf31,item_category_tfidf32,item_category_tfidf33,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37,location
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,west
1,2,5,5233,"Need for Speed Rivals (Essentials) [PS3, русск...",19,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,west
2,15,5,4872,"Minecraft. Playstation 3 Edition [PS3, русская...",19,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,west
3,21,5,5002,"NBA 2K16 [PS3, русская документация]",19,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,west
4,23,5,5814,"Plants vs. Zombies Garden Warfare [PS3, русска...",19,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,west


In [61]:
dummy_loc_df =pd.get_dummies(test_df['location'])
dummy_loc_df.head()

,Digital,Online,east,south,west
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1


In [62]:
test_df = pd.concat([test_df, dummy_loc_df],axis=1)
test_df = test_df.drop(['location'],axis=1)
test_df.head()

,ID,shop_id,item_id,item_name,item_category_id,item_tfidf0,item_tfidf1,item_tfidf2,item_tfidf3,item_tfidf4,...,item_category_tfidf33,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37,Digital,Online,east,south,west
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1
1,2,5,5233,"Need for Speed Rivals (Essentials) [PS3, русск...",19,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1
2,15,5,4872,"Minecraft. Playstation 3 Edition [PS3, русская...",19,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1
3,21,5,5002,"NBA 2K16 [PS3, русская документация]",19,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1
4,23,5,5814,"Plants vs. Zombies Garden Warfare [PS3, русска...",19,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1


In [63]:
test_df = test_df.drop(['item_name'],axis=1)
test_df = test_df.drop(['item_category_name'],axis=1)
test_df = test_df.drop(['item_category_id'],axis=1)
test_df = test_df.drop(['shop_name'],axis=1)
test_df.head()

,ID,shop_id,item_id,item_tfidf0,item_tfidf1,item_tfidf2,item_tfidf3,item_tfidf4,item_tfidf5,item_tfidf6,...,item_category_tfidf33,item_category_tfidf34,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37,Digital,Online,east,south,west
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1
1,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1
2,15,5,4872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1
3,21,5,5002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1
4,23,5,5814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1


In [64]:
test_df['month'] =11
test_df['year'] = 2015
test_df['date_block_num']= 34
# test_df["price_fluctuation"] = 0
test_df["Outbound"] = 0
test_df = test_df.sort_values("ID")
test_df.head()

,ID,shop_id,item_id,item_tfidf0,item_tfidf1,item_tfidf2,item_tfidf3,item_tfidf4,item_tfidf5,item_tfidf6,...,item_category_tfidf37,Digital,Online,east,south,west,month,year,date_block_num,Outbound
0,0,5,5037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,1,11,2015,34,0
5880,1,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,1,11,2015,34,0
1,2,5,5233,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,1,11,2015,34,0
34104,3,5,5232,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,1,11,2015,34,0
40320,4,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0,0,0,1,11,2015,34,0


In [65]:
set(test_df.columns)

{'Digital',
 'ID',
 'Online',
 'Outbound',
 'date_block_num',
 'east',
 'item_category_tfidf0',
 'item_category_tfidf1',
 'item_category_tfidf10',
 'item_category_tfidf11',
 'item_category_tfidf12',
 'item_category_tfidf13',
 'item_category_tfidf14',
 'item_category_tfidf15',
 'item_category_tfidf16',
 'item_category_tfidf17',
 'item_category_tfidf18',
 'item_category_tfidf19',
 'item_category_tfidf2',
 'item_category_tfidf20',
 'item_category_tfidf21',
 'item_category_tfidf22',
 'item_category_tfidf23',
 'item_category_tfidf24',
 'item_category_tfidf25',
 'item_category_tfidf26',
 'item_category_tfidf27',
 'item_category_tfidf28',
 'item_category_tfidf29',
 'item_category_tfidf3',
 'item_category_tfidf30',
 'item_category_tfidf31',
 'item_category_tfidf32',
 'item_category_tfidf33',
 'item_category_tfidf34',
 'item_category_tfidf35',
 'item_category_tfidf36',
 'item_category_tfidf37',
 'item_category_tfidf4',
 'item_category_tfidf5',
 'item_category_tfidf6',
 'item_category_tfidf7',
 

In [66]:
set(test_df.columns)

{'Digital',
 'ID',
 'Online',
 'Outbound',
 'date_block_num',
 'east',
 'item_category_tfidf0',
 'item_category_tfidf1',
 'item_category_tfidf10',
 'item_category_tfidf11',
 'item_category_tfidf12',
 'item_category_tfidf13',
 'item_category_tfidf14',
 'item_category_tfidf15',
 'item_category_tfidf16',
 'item_category_tfidf17',
 'item_category_tfidf18',
 'item_category_tfidf19',
 'item_category_tfidf2',
 'item_category_tfidf20',
 'item_category_tfidf21',
 'item_category_tfidf22',
 'item_category_tfidf23',
 'item_category_tfidf24',
 'item_category_tfidf25',
 'item_category_tfidf26',
 'item_category_tfidf27',
 'item_category_tfidf28',
 'item_category_tfidf29',
 'item_category_tfidf3',
 'item_category_tfidf30',
 'item_category_tfidf31',
 'item_category_tfidf32',
 'item_category_tfidf33',
 'item_category_tfidf34',
 'item_category_tfidf35',
 'item_category_tfidf36',
 'item_category_tfidf37',
 'item_category_tfidf4',
 'item_category_tfidf5',
 'item_category_tfidf6',
 'item_category_tfidf7',
 

In [67]:
train_df.head()

,date_block_num,shop_id,item_id,month,year,shop_name_tfidf0,shop_name_tfidf1,shop_name_tfidf2,shop_name_tfidf3,shop_name_tfidf4,...,item_category_tfidf35,item_category_tfidf36,item_category_tfidf37,Digital,Online,Outbound,east,south,west,ID
0,0,59,22154,01,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,1,0
1,0,25,22154,01,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,1,1
2,0,25,22154,01,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,1,2
3,0,25,22154,01,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,1,3
4,0,25,22154,01,2013,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,1,4


# Evaluation

In [68]:
y_data.shape

(2935849, 1)

In [69]:
test_data = test_df.values

In [70]:
test_data.shape

(214200, 148)

In [71]:
x_data = train_df.values

In [72]:
x_data.shape

(2935849, 148)

In [73]:
from sklearn.linear_model import Lasso,LinearRegression
lr_simple = LinearRegression().fit(x_data, y_data)
lr_lasso = Lasso().fit(x_data, y_data)

In [74]:
y_hat_simple = lr_simple.predict(test_data)

In [75]:
simple_lr_result = pd.DataFrame(y_hat_simple, columns=["item_cnt_month"])
simple_lr_result.index.name = 'ID'
simple_lr_result.to_csv("simple_lr_result.csv", sep=",")
simple_lr_result.head()

,item_cnt_month
ID,
0,5.493307e+08
1,5.493534e+08
2,5.493761e+08
3,5.493989e+08
4,5.494216e+08


In [76]:
y_hat_lasso = lr_lasso.predict(test_data)

In [77]:
lasso_lr_result = pd.DataFrame(y_hat_lasso, columns=["item_cnt_month"])
lasso_lr_result.index.name = 'ID'
lasso_lr_result.to_csv("lasso_lr_result.csv", sep=",")
lasso_lr_result.head()

,item_cnt_month
ID,
0,1.215017
1,1.214375
2,1.214572
3,1.214574
4,1.214493


In [78]:
lasso_lr_result.to_csv("hypo_text_loc_result.csv", sep=",")